Welcome!

This is a daily report, on the performance of specific exchange rates. It is to be ran at market close, to determine potential courses of action.

In [2]:
###Daily report

import pyodbc
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests
import pandas_ta as ta
import pyodbc

global dictdate
dictdate = {
    "AXS" : "2020-11-04",
    "ENS" : "2021-11-09",
    "NEXO": "2018-05-01",
    "DOT" : "2020-08-20",
    "XRP" : "2017-11-09",
    "ETH" : "2017-11-09",
    "ARB11841" : "2023-03-23",
    "TIA22861" : "2023-10-31",
    "ADA" : "2017-11-09",
    "HBAR" : "2019-09-17",
    "SOL" : "2020-04-10",
    "NEAR" : "2020-10-14",
    "RUNE" : "2019-07-23",
    "FTM" : "2018-10-30",
    "LINK" : "2017-11-09",
    "BCH" : "2017-11-09",
    "AVAX" : "2020-07-13",
    "ORDI" : "2023-05-08",
    "OP" : "2022-03-14",
    "DOGE" : "2017-11-09",
    "BONK" : "2022-12-30",
    "MATIC" : "2019-04-28",
    "AI" : "2021-09-22",
    "LTC" : "2014-09-17",
    "BNB" : "2017-11-09",
    "ETC" : "2017-11-09",
    "JUP" : "2017-11-09",
    "XMR" : "2017-11-09",
    "FIL" : "2017-12-13",
    "DYDX" : "2021-11-08",
    "ATOM" : "2019-03-14",
    "EOS" : "2017-11-09",
    "TRB" : "2019-11-19",
    "FTM" : "2018-10-30",
    "JTO" : "2023-12-07",
    "GALA" : "2020-09-18",
    "BLUR" : "2023-02-14"
}

def dataImport(token):

    global creation
    global current
    global activetime

    if token in dictdate.keys():
        datestr = dictdate[token]
    elif token == "ARB":
        datestr = "2023-03-23"
        token = "ARB11841"
    elif token == "TIA":
        datestr = "2023-10-31"
        token = "TIA22861"
    else:
        datestr = input("Not in our database! Enter a date (YYYY-MM-DD): ")
        print ("Don't forget to add the date to the dictionary!")

    creation = datetime.strptime(datestr, "%Y-%m-%d")
    current = datetime.today()
    activetime = (current - creation).days

    # Connection parameters - replace with your values
    server = 'PERSONALDEVICEG' 
    database = 'models'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')

    cursor = cnxn.cursor()

    # Set up the table and the auto procedure

    try:
        # It's more common to use separate execute calls for each SQL statement, but here's a try for batch execution
        cursor.execute(f"""
            IF OBJECT_ID('{token}daily', 'P') IS NOT NULL
                DROP PROCEDURE {token}daily;
        """)

        cursor.execute(f"""            
            IF OBJECT_ID('{token}HistoricalPrices', 'U') IS NOT NULL
                DROP TABLE {token}HistoricalPrices;
        """)

        cursor.execute(f"""
            CREATE TABLE {token}HistoricalPrices (
                [Date] DATE,
                [Open] DECIMAL(10, 6),
                [High] DECIMAL(10, 6),
                [Low] DECIMAL(10, 6),
                [Close] DECIMAL(10, 6),
                [Adj Close] DECIMAL(10, 6),
                [Volume] BIGINT
            );
        """)

        cursor.execute(f"""
            CREATE PROCEDURE {token}daily 
                @dates DATE,
                @opens DECIMAL(10, 6),
                @highs DECIMAL(10, 6),
                @lows DECIMAL(10, 6),
                @closes DECIMAL(10, 6),
                @adjcloses DECIMAL(10, 6),
                @volumes BIGINT
            AS BEGIN
                INSERT INTO {token}HistoricalPrices (
                    [Date],
                    [Open],
                    [High],
                    [Low],
                    [Close],
                    [Adj Close],
                    [Volume]
                )
                VALUES (
                    @dates,
                    @opens,
                    @highs,
                    @lows,
                    @closes,
                    @adjcloses,
                    @volumes
                );
            END;
        """)

        cnxn.commit()

    except Exception as e:
        print(f'Preliminary SQL execution failed: {e}')
        cursor.close()
        cnxn.close()
        return


    # Read data from Yahoo! Finance

    today_date = datetime.today().strftime('%Y-%m-%d')

    df = yf.download(f"{token}-USD", start=datestr, end=today_date)

    df.reset_index(inplace=True)

    # Loop through the DataFrame and insert each row
    for index, row in df.iterrows():
        cursor.execute(f"EXEC {token}daily @dates = ?, @opens = ?, @highs = ?, @lows = ?, @closes = ?, @adjcloses = ?, @volumes = ?", 
                       row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume'])

    # Commit the transaction
    try :
        cnxn.commit()
    except:
        print ('Data import failed!')

    # Close the connection
    cursor.close()
    cnxn.close()



# Function to calculate the daily percentage change in price and volume
def calculate_price_volume_change(df):
    # Calculate daily percentage change in price
    df['Price_Change'] = df['Close'].pct_change() * 100
    
    # Calculate daily percentage change in volume
    df['Volume_Change'] = df['Volume'].pct_change() * 100
    
    return df

# Function to identify potential pump-and-dump activities based on predefined thresholds
def identify_potential_pump_and_dump(df, price_change_threshold, volume_change_threshold):
    # Identify potential pump-and-dump activities
    df['Potential_Pump_and_Dump'] = (df['Price_Change'] > price_change_threshold) & (df['Volume_Change'] > volume_change_threshold)
    
    return df

# Function to conduct market context analysis incorporating pump-and-dump analysis
def conduct_market_context_analysis(df):
    # Initialize market context column
    df['Market_Context'] = 'Neutral'
    
    # Update market context based on pump-and-dump analysis
    df.loc[df['Potential_Pump_and_Dump'], 'Market_Context'] = 'Potential Pump-and-Dump'
    
    return df

def emotional(token, index):

    pd.set_option('display.max_columns', None)

    cnxn = connectionEst()

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    if token == "ARB":
        token = "ARB11841"

    if token == "TIA":
        token = "TIA22861"


    # Define thresholds for identifying potential pump-and-dump activities
    price_change_threshold = 10  # Percentage change in price threshold
    volume_change_threshold = 20  # Percentage change in volume threshold

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    # Calculate price and volume changes
    df = calculate_price_volume_change(df)

    # Identify potential pump-and-dump activities
    df = identify_potential_pump_and_dump(df, price_change_threshold, volume_change_threshold)

    # Conduct market context analysis
    df = conduct_market_context_analysis(df)

    df['Index'] = index

    # Display the DataFrame with updated market context
    return df.tail(1).sort_values(by='Date', ascending = False)

def connectionEst():

    global cnxn

    # Connection parameters - replace with your values
    server = 'PERSONALDEVICEG' 
    database = 'models'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')
    return cnxn


def calculate_technical_indicators(df, index, period=20, num_std=2):
    # Assuming df has 'Close', 'High', and 'Low' columns

    # Convert 'Date' to datetime and set as index

    # Calculate Moving Averages
    df['SMA_20'] = ta.sma(df['Close'], length=period)
    df['EMA_20'] = ta.ema(df['Close'], length=period)
   
    # Bollinger Bands
    df['SMA'] = df['Close'].rolling(window=period).mean()  # Simple Moving Average
    df['STD'] = df['Close'].rolling(window=period).std()  # Standard Deviation
    df['Upper_Band'] = df['SMA'] + (df['STD'] * num_std)
    df['Lower_Band'] = df['SMA'] - (df['STD'] * num_std)

    # RSI
    df['RSI_14'] = ta.rsi(df['Close'], length=14)

    # MACD
    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    try:
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_Signal'] = macd['MACDs_12_26_9']
        df['MACD_Histogram'] = macd['MACDh_12_26_9']
    except:
        df['MACD'] = "Not available"
        df['MACD_Signal'] = "Not available"
        df['MACD_Histogram'] = "Not available"
    
    # Identifying bullish or bearish trends
    try:
        df['Trend'] = 'Neutral'
        df.loc[df['Close'] > df['SMA_20'], 'Trend'] = 'Bullish'
        df.loc[df['Close'] < df['SMA_20'], 'Trend'] = 'Bearish'
    except:
        df['Trend'] = "Not available"


    # Volume confirmation
    df['Volume_Confirmation'] = False
    df.loc[(df['Volume'].shift(1) < df['Volume']) & 
           ((df['Trend'] == 'Bullish') & (df['Close'] > df['Close'].shift(1)) | 
            (df['Trend'] == 'Bearish') & (df['Close'] < df['Close'].shift(1))), 'Volume_Confirmation'] = True

    # Bollinger Bands for volatility and price position
    df['Bollinger_Signal'] = 'Neutral'
    df.loc[df['Close'] > df['Upper_Band'], 'Bollinger_Signal'] = 'Overbought'
    df.loc[df['Close'] < df['Lower_Band'], 'Bollinger_Signal'] = 'Oversold'

    # RSI for momentum
    df['RSI_Signal'] = 'Neutral'
    df.loc[df['RSI_14'] > 70, 'RSI_Signal'] = 'Overbought'
    df.loc[df['RSI_14'] < 30, 'RSI_Signal'] = 'Oversold'

    # MACD for trend changes
    df['MACD_Signal_Char'] = 'Neutral'
    df.loc[df['MACD'] > df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bullish'
    df.loc[df['MACD'] < df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bearish'

    # Price action relative to moving averages for potential reversals
    try:
        df['Price_MA_Relation'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'].shift(1) < df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Up'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'].shift(1) > df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Down'
    except:
         df['Price_MA_Relation'] = 'Not available'

    try:
        df['Trend_Volume'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bullish with Volume Increase'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bearish with Volume Increase'
    except:
        df['Trend_Volume'] = 'Not available'
    
    # Bollinger Bands and Volume for Market Extremes
    df['Bollinger_Volume'] = 'Neutral'
    df.loc[(df['Close'] > df['Upper_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Overbought with Volume Increase'
    df.loc[(df['Close'] < df['Lower_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Oversold with Volume Increase'
    
    # RSI and Volume for Momentum Confirmation
    df['RSI_Volume'] = 'Neutral'
    df.loc[(df['RSI_14'] > 70) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Overbought with High Volume'
    df.loc[(df['RSI_14'] < 30) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Oversold with High Volume'
    
    # MACD, Volume, and Market Reversals
    df['MACD_Signal_Volume'] = 'Neutral'
    df.loc[(df['MACD'] > df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bullish Crossover with Volume Increase'
    df.loc[(df['MACD'] < df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bearish Crossover with Volume Increase'
    
    # Contextual Analysis Based on Historical Highs/Lows
    try:
        df['Market_Context'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Close'] < df['Low'].min() * 1.10), 'Market_Context'] = 'Bullish Near Historical Lows'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Close'] > df['High'].max() * 0.90), 'Market_Context'] = 'Bearish Near Historical Highs'
    except:
        df['Market_Context'] = 'Not available'

    df ['Index'] = index

    return df.tail(1).sort_values(by='Date', ascending = False)


def technical(token, index):

    pd.set_option('display.max_columns', None)

    cnxn = connectionEst()

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    if df.empty:
        return

    return calculate_technical_indicators(df, index)


emotionalFrame = None
technicalFrame = None
appendix = None
new = None
index = None

for k in dictdate.keys():

    dataImport(k)

for k in dictdate.keys():

    if emotionalFrame is None:
        new = emotional (k, k)
        emotionalFrame = new
    else:
        appendix = emotional(k, k)
        emotionalFrame = pd.concat((emotionalFrame, appendix), axis = 0)

for k in dictdate.keys():

    if technicalFrame is None:
        new = technical(k, k)
        technicalFrame = new
    else:
        appendix = technical(k, k)
        technicalFrame = pd.concat((technicalFrame, appendix), axis = 0)

complete = pd.merge(emotionalFrame, technicalFrame, on = 'Index')

complete

    

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

...

<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are no

<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are no

<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are no

<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-2-ec1958dca272>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


,Date_x,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Price_Change,Volume_Change,Potential_Pump_and_Dump,Market_Context_x,Index,Date_y,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,SMA_20,EMA_20,SMA,STD,Upper_Band,Lower_Band,RSI_14,MACD,MACD_Signal,MACD_Histogram,Trend,Volume_Confirmation,Bollinger_Signal,RSI_Signal,MACD_Signal_Char,Price_MA_Relation,Trend_Volume,Bollinger_Volume,RSI_Volume,MACD_Signal_Volume,Market_Context_y
0,2024-06-06,8.613020,8.697354,8.385620,8.433172,8.433172,54664343,-2.088095,-36.962275,False,Neutral,AXS,2024-06-06,8.613020,8.697354,8.385620,8.433172,8.433172,54664343,7.850865,7.881686,7.850865,0.367253,8.585372,7.116358,59.487774,0.203621,0.115749,8.787186e-02,Bullish,False,Neutral,Neutral,Bullish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
1,2024-06-06,25.264006,25.652092,23.842999,24.053110,24.053110,94818250,-3.550290,-19.832567,False,Neutral,ENS,2024-06-06,25.264006,25.652092,23.842999,24.053110,24.053110,94818250,23.448359,23.294915,23.448359,3.754635,30.957629,15.939089,56.248530,2.240070,2.489140,-2.490708e-01,Bullish,False,Neutral,Neutral,Bearish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
2,2024-06-06,1.482461,1.502117,1.466940,1.482463,1.482463,7729841,0.000135,14.515722,False,Neutral,NEXO,2024-06-06,1.482461,1.502117,1.466940,1.482463,1.482463,7729841,1.428476,1.417869,1.428476,0.043437,1.515349,1.341603,65.363184,0.040421,0.039968,4.531380e-04,Bullish,True,Neutral,Neutral,Bullish,Neutral,Bullish with Volume Increase,Neutral,Neutral,Bullish Crossover with Volume Increase,Neutral
3,2024-06-06,7.248750,7.279625,7.068440,7.142014,7.142014,154128107,-1.472176,-9.970235,False,Neutral,DOT,2024-06-06,7.248750,7.279625,7.068440,7.142014,7.142014,154128107,7.247596,7.163746,7.247596,0.227202,7.702001,6.793192,49.153588,-0.010396,0.000733,-1.112904e-02,Bearish,False,Neutral,Neutral,Bearish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
4,2024-06-06,0.525706,0.527604,0.517547,0.521578,0.521578,1194300808,-0.786173,3.489912,False,Neutral,XRP,2024-06-06,0.525706,0.527604,0.517547,0.521578,0.521578,1194300808,0.525640,0.523243,0.525640,0.008434,0.542508,0.508772,48.476282,-0.001031,-0.000956,-7.483892e-05,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
5,2024-06-06,3864.263428,3878.052246,3761.775391,3811.605957,3811.605957,13606583873,-1.362616,-12.102367,False,Neutral,ETH,2024-06-06,3864.263428,3878.052246,3761.775391,3811.605957,3811.605957,13606583873,3715.768835,3671.349114,3715.768835,217.688263,4151.145362,3280.392309,62.756544,152.767496,162.802453,-1.003496e+01,Bullish,False,Neutral,Neutral,Bearish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
6,2024-06-06,1.122950,1.123754,1.077755,1.088713,1.088713,226273579,-3.048586,-19.456279,False,Neutral,ARB11841,2024-06-06,1.122950,1.123754,1.077755,1.088713,1.088713,226273579,1.134718,1.118514,1.134718,0.062684,1.260086,1.009351,46.033242,0.001909,0.007588,-5.679471e-03,Bearish,False,Neutral,Neutral,Bearish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
7,2024-06-06,10.468363,10.527226,10.209706,10.430121,10.430121,69699245,-0.365310,-44.288197,False,Neutral,TIA22861,2024-06-06,10.468363,10.527226,10.209706,10.430121,10.430121,69699245,10.154487,10.344029,10.154487,0.899570,11.953627,8.355348,51.426219,0.272020,0.212673,5.934629e-02,Bullish,False,Neutral,Neutral,Bullish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
8,2024-06-06,0.461175,0.461865,0.452339,0.458102,0.458102,281512944,-0.666341,-6.424282,False,Neutral,ADA,2024-06-06,0.461175,0.461865,0.452339,0.458102,0.458102,281512944,0.463960,0.459246,0.463960,0.015491,0.494942,0.432978,48.051291,-0.003900,-0.004807,9.067054e-04,Bearish,False,Neutral,Neutral,Bullish,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
9,2024-06-06,0.104454,0.105027,0.100215,0.101416,0.101416,52088501,-2.908457,-11.949100,False,Neutral,HBAR,2024-06-06,0.104454,0.105027,0.100215,0.101416,0.101416,52088501,0.106369,0.10